## Tensor Types in TensorFlow

(You can also read this article on our website, [Easy-TensorFlow](http://www.easy-tensorflow.com/basics/tensor-types))

In the previous post, we read about the concepts of __Graph__ and __Session__ which describes the way the data flows in TensorFlow. One of the first questions you might have while learning a new framework is of any new data structure that should used. TensorFlow does have its own data structure for the purpose of performance and ease of use. Tensor is the data structure used in Tensorflow (remember TensorFlow is the flow of tensors in a computational graph) and it is at the core of TensorFlow. TensorFlow programs use a tensor data structure to represent all data — only tensors are passed between operations in the computation graph. You can think of a TensorFlow tensor as an n-dimensional array or list.

In this tutorial, we'll take a look at some of the __Tensor Types__ used in TensorFlow. The speciall ones commonly used in creating neural network models are namely ___Constant___, ___Variable___, and ___Placeholder___. 

This will also help us to shed some light on some of the points and questions left unanswered in the previous post.

Remember that we need to import the TensorFlow library at the very beginning of our code using the line:

In [1]:
import tensorflow as tf

## 1. Constant 

As the name speaks for itself, __Constants__ are used as constants. They create a node that takes value and it does not change. You can simply create a constant tensor using __tf.constant__. It accepts the five arguments:


```python
tf.constant(value, dtype=None, shape=None, name='Const')
```

Now let's take a look at a very simple example.

### Example 1:
Let's create two constants and add them together. Constant tensors can simply be defined with a value:

In [2]:
# create graph
a = tf.constant(2)
b = tf.constant(3)
c = a + b

print(c)

tf.Tensor(5, shape=(), dtype=int32)


Constants can be defined with different types (integer, float, etc.) and shapes (vectors, matrices, etc.). The next example has one constant with type 32bit float and another constant with shape 2X2.


### Example 2:

In [3]:
s = tf.constant(2.3, dtype=tf.float32)
m = tf.constant([[1, 2], [3, 4]], dtype=tf.int16)

print(s)
print(m)

tf.Tensor(2.3, shape=(), dtype=float32)
tf.Tensor(
[[1 2]
 [3 4]], shape=(2, 2), dtype=int16)


## 2. Variable

Variables are stateful nodes which output their current value; meaning that they can retain their value over multiple executions of a graph. They have a number of useful features such as:

- They can be __saved__ to your disk during and after training. This allows people from different companies and groups to collaborate as they can save, restore and send over their model parameters to other people. (read more on [Save and Restore Tutorial](https://github.com/easy-tensorflow/easy-tensorflow/blob/master/1_TensorFlow_Basics/Tutorials/4_Save_and_Restore.ipynb))
- By default, gradient updates (used in all neural networks) will apply to all variables in your graph. In fact, variables are the things that you want to tune in order to minimize the loss. 

These two features make variables suitable to be used as the network parameters (i.e. weights and biases). You might ask, what are the differences between variables and constants? Well there are two major differences:

1. Constants are (guess what!), constants. As their name states, their value doesn't change. We'd usually need our network parameters to be updated and that's where the __variable__ comes into play.

2. Constants are stored in the graph definition which makes them memory-expensive. In other words, constants with millions of entries makes the graph slower and resource intensive.


Again, it's important to remember that creating a variables is an operation (look at the Fig. 2 of the first tutorial for a quick recap). We execute these operations in the session and get the output value of the operations.

To create a variable, we should use __tf.Variable__ as:

```python
w = tf.Variable(<initial-value>, initial_value=None, trainable=None, name=None, dtype=None)
```

For example we can create scalar or matrix variables as follows:

In [4]:
s = tf.Variable(2, name="scalar") 
m = tf.Variable([[1, 2], [3, 4]], name="matrix") 
W = tf.Variable(tf.zeros([784,10]))

Variable __W__ defined above creates a matrix with 784 rows and 10 columns which will be initialized with zeros. This can be used as a weight matrix of a feed-forward neural network (or even in a linear regression model) from a layer with 784 neuron to a layer with 10 neuron. We'll see more of this later in this turorial.

__*Note:__ We use tf.Variable() with uppercase "V", and tf.constant with lowercase "c". You don't necessarily need to know the reason, but it's simply because tf.constant is an op, while tf.Variable is a class with multiple ops.

## 3. tf.function

TensorFlow 2.0 introduces a new way to create graphs in TensorFlow using __tf.function__. This is a very powerful feature that allows you to convert regular Python code into a callable TensorFlow graph function. This is very useful when you want to optimize your code and make it run faster.

We can use __tf.function__ as a decorator to convert a Python function into a callable TensorFlow graph function. Let's take a look at a simple example to understand how it works.

In [5]:
@tf.function
def matmul(x, y):
    return tf.matmul(x, y)

x = tf.constant([[1., 2.], [3., 4.], [5., 6.]], dtype=tf.float32)
y = tf.constant([[1., 2.], [3., 4.]], dtype=tf.float32)

print(matmul(x, y))


tf.Tensor(
[[ 7. 10.]
 [15. 22.]
 [23. 34.]], shape=(3, 2), dtype=float32)


So far so good? To make it more interesting and challenging lets get our hands dirty!

## Creating a Neural Network

Now, we have all the required materials to start building a toy feed-forward neural network with one hidden layer and 200 hidden units (neurons). The computational graph in Tensorflow will be:

<img src="files/files/2_5.png" width="300" height="600" >
___Fig5. ___ Schematic of the graph for one layer of the neural network

How many operations (or nodes) do you see in this graph? Six! right? The three circles (X, W, b) and three rectangles. We'll go through each of them and will discuss the best way to implement it.

Let's start with the input, X. This can be an input of any type, such as images, signals, etc. The general approach is to feed all inputs to the network and train the trainable parameters (here, W and b) by backpropagating the error signal. Ideally, you need to feed all inputs together, compute the error, and update the parameters. This process is called "Gradient Descent".

*Side Note: In real-world problems, we have thousands and millions of inputs which makes gradient descent computationally expensive. That's why we split the input set into several shorter pieces (called mini-batch) of size B (called mini-batch size) inputs, and feed them one by one. This is called "Stochastic Gradient Descent". The process of feeding each mini-batch of size B to the network, back-propagating errors, and updating the parameters (weights and biases) is called an iteration.

Here, we have a feed-forward neural network, and let's assume we have 500 input samples and each sample is of size 784 (similar to 28x28 images of MNIST data). The input tensor can be written as:

In [6]:
# create the input tensor
X = tf.random.normal([500, 784], 0, 1, tf.float32)

Now we need to create the weight and bias tensors. The weight tensor is of size 784x200 and the bias tensor is of size 200. We can create these tensors using __tf.Variable__ as:

In [7]:
W = tf.Variable(tf.random.normal([784, 200], stddev=0.01))
b = tf.Variable(tf.zeros([200]))

Okay, we are all set. The created graph looks like this:
<img src="files/files/2_6.png" width="400" height="800" >
___Fig6. ___ Data flow graph of the neural network created in Tensorflow

But how can we visualize this graph? How do you create this figure? That's the magic of __Tensorboard__. It's thoroughly explained in our next article.

Now let's create a function to compute the output of the neural network:

In [8]:
@tf.function
def neural_network(X, W, b):
    # Create the operations
    x_w = tf.matmul(X, W, name="MatMul")
    x_w_b = tf.add(x_w, b, name="Add")
    h = tf.nn.relu(x_w_b, name="ReLU")
    return h

# get the output of the neural network
h = neural_network(X, W, b)
print(h)

tf.Tensor(
[[0.         0.2894448  0.         ... 0.         0.03060435 0.        ]
 [0.1961359  0.         0.         ... 0.         0.         0.20836228]
 [0.10028808 0.49761808 0.         ... 0.4213311  0.         0.2812415 ]
 ...
 [0.         0.         0.         ... 0.38792145 0.         0.7286719 ]
 [0.         0.         0.         ... 0.30122933 0.29370958 0.04947911]
 [0.07269141 0.         0.36451468 ... 0.         0.         0.        ]], shape=(500, 200), dtype=float32)


Running this code will print out h$_{[500, 200]}$ which are the outputs of 200 hidden units in response to 500 images; i.e. 200 features extracted from 500 images.

We'll continue constructing the loss function and creating the optimizer operations in the next articles. However, we need to learn Tensorboard first to use its amazing features in our neural network code.

I hope this post has helped you to understand how to use different __Tensor Types__ in TensorFlow. Thank you so much for reading! If you have any questions, feel free to leave a comment in our [webpage](http://www.easy-tensorflow.com/basics/tensor-types). You can also send us feedback through the [__contacts__](http://www.easy-tensorflow.com/contacts) page.